## Explore and cluster the neighborhoods in Toronto

Import libraries

In [145]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import geocoder

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


In [149]:
df = pd.read_csv('Ass2.csv')
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


Get geographical coordinates for Toronto

In [150]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

toronto_coor = get_latlng('')
toronto_coor

[43.648690000000045, -79.38543999999996]

According to assignment instructions, work with only boroughs that contain the word Toronto 

In [151]:
df_toronto = df[ df.Borough.str.contains('Toronto') ]
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295410
41,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.667985,-79.314642
43,M4M,East Toronto,Studio District,43.662766,-79.334830
44,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090
45,M4P,Central Toronto,Davisville North,43.712755,-79.388514
46,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
47,M4S,Central Toronto,Davisville,43.702765,-79.385769
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.690505,-79.382973
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686003,-79.402335


Plot the data on a map

In [152]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_coor[0], toronto_coor[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Load FourSquare account information

In [153]:
CLIENT_ID = 'FMR3OGXQDOSLEBPX30G4YWRNANUJWHIWWIK0A0ARKJDRCT5Y' # your Foursquare ID
CLIENT_SECRET = 'FS2N20SQTXFROYJIGGXV032YMHCQJZHXL2DW0VSCGMUJQPS4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [154]:
df_toronto = df_toronto.reset_index().drop('index', axis=1)
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [155]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67653131600008, -79.29540999999995.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [156]:
# type your answer here
VERSION = '20180605'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=FMR3OGXQDOSLEBPX30G4YWRNANUJWHIWWIK0A0ARKJDRCT5Y&client_secret=FS2N20SQTXFROYJIGGXV032YMHCQJZHXL2DW0VSCGMUJQPS4&ll=43.67653131600008,-79.29540999999995&v=20180605&radius=500&limit=100'

Send the GET request and examine the results

In [157]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3b7ef29fb6b726606cfabb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.68103132050008,
    'lng': -79.2891996964911},
   'sw': {'lat': 43.67203131150008, 'lng': -79.3016203035088}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e77e3861f6ecf8d3648300c',
       'name': 'Starbucks',
       'location': {'address': '637 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.67879837444001,
        'lng': -79.2980449760153,
        'labeledLatLngs': [{'label': 

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab

In [158]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [159]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.678798,-79.298045
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Dip 'n Sip,Coffee Shop,43.678897,-79.297745
3,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [160]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [161]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [162]:
toronto_venues = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction 

#### Let's check the size of the resulting dataframe

In [163]:
print(toronto_venues.shape)
toronto_venues.head()

(1715, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.29541,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676531,-79.29541,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676531,-79.29541,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
3,The Beaches,43.676531,-79.29541,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.683262,-79.35512,Dairy Queen,43.684223,-79.357062,Fast Food Restaurant


Let's check how many venues were returned for each neighborhood

In [164]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",66,66,66,66,66,66
Berczy Park,62,62,62,62,62,62
"Brockton, Exhibition Place, Parkdale Village",53,53,53,53,53,53
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,98,98,98,98,98,98
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,9,9,9,9,9,9


#### Let's find out how many unique categories can be curated from all the returned venues

In [165]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 211 uniques categories.


### Analyze Each Neighborhood

In [166]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [167]:
toronto_onehot.shape

(1715, 211)

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [168]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.015152,0.0000,0.0000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.00
2,Berczy Park,0.016129,0.0000,0.0000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,...,0.018868,0.018868,0.000000,0.00,0.037736,0.000000,0.000000,0.000000,0.000000,0.00
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.020000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.000000,0.0000,0.0000,0.010204,0.000000,0.000000,0.010204,0.010204,0.000000,...,0.010204,0.000000,0.010204,0.00,0.010204,0.010204,0.010204,0.010204,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.000000,0.000000,0.010638,0.021277,0.000000,0.010638,...,0.000000,0.010638,0.000000,0.00,0.042553,0.000000,0.042553,0.010638,0.000000,0.00
8,Christie,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.000000,0.0125,0.0125,0.012500,0.000000,0.000000,0.000000,0.000000,0.012500,...,0.012500,0.000000,0.000000,0.00,0.000000,0.012500,0.012500,0.000000,0.012500,0.00


##### Let's confirm the new size

In [169]:
toronto_grouped.shape

(37, 211)

##### Let's print each neighborhood along with the top 5 most common venues

In [170]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                Hotel  0.07
1                 Café  0.06
2          Coffee Shop  0.05
3  American Restaurant  0.04
4                  Bar  0.03


----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.06
2                 Pub  0.05
3                Café  0.05
4                Park  0.05


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.06
2  Cocktail Bar  0.05
3          Café  0.03
4           Pub  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                     Bar  0.06
2                    Café  0.06
3  Furniture / Home Store  0.06
4      Italian Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.07
1    

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [171]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [172]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Café,Coffee Shop,American Restaurant,Gastropub,Japanese Restaurant,Bar,Steakhouse,Restaurant,Deli / Bodega
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Coffee Shop,Italian Restaurant,Restaurant,Park,Pub,Café,Bakery,Speakeasy,Sandwich Place,Diner
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Hotel,Café,Italian Restaurant,Cheese Shop,Pub,Bakery
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Bar,Café,Italian Restaurant,Hotel,Gift Shop,Sandwich Place,Vegetarian / Vegan Restaurant,French Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Steakhouse,Restaurant,Japanese Restaurant,Hotel,American Restaurant,Thai Restaurant,Concert Hall
5,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Italian Restaurant,Café,Bakery,Market,Pizza Place,Breakfast Spot,Japanese Restaurant,Jewelry Store
6,Central Bay Street,Coffee Shop,Clothing Store,Café,Plaza,Tea Room,Chinese Restaurant,Spa,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Ice Cream Shop,Bakery,Chinese Restaurant,Art Gallery
8,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Playground,Event Space,Exhibit,Falafel Restaurant,Women's Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Sushi Restaurant,Pub,Gastropub,Fast Food Restaurant,Dance Studio


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [173]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [174]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295410,1,Coffee Shop,Pub,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120,1,Fast Food Restaurant,Park,Bus Line,Grocery Store,Discount Store,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Farmers Market
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.667985,-79.314642,1,Park,Pet Store,Italian Restaurant,Burrito Place,Burger Joint,Fish & Chips Shop,Sandwich Place,Liquor Store,Board Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.662766,-79.334830,1,Diner,Bakery,Italian Restaurant,Café,Bar,Sushi Restaurant,American Restaurant,Coffee Shop,Boutique,Juice Bar
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090,1,Bus Line,Swim School,Women's Store,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Finally, let's visualize the resulting clusters

In [175]:
# create map
map_clusters = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [176]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Sushi Restaurant,Pub,Gastropub,Fast Food Restaurant,Dance Studio
22,Central Toronto,0,Park,Women's Store,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
23,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pizza Place,Indian Restaurant,Mexican Restaurant,Burger Joint,Restaurant,Liquor Store,Pub
26,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Restaurant,Park,Pub,Café,Bakery,Speakeasy,Sandwich Place,Diner
28,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Steakhouse,Restaurant,Bar,Deli / Bodega,Japanese Restaurant,Gastropub
30,West Toronto,0,Park,Bakery,Pet Store,Discount Store,Pool,Middle Eastern Restaurant,Café,Fast Food Restaurant,Furniture / Home Store,Supermarket
31,West Toronto,0,Bar,Restaurant,Cocktail Bar,Asian Restaurant,Men's Store,Coffee Shop,Pizza Place,Bakery,Vietnamese Restaurant,Wine Bar


### Cluster 2

In [177]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Coffee Shop,Pub,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,East Toronto,1,Fast Food Restaurant,Park,Bus Line,Grocery Store,Discount Store,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Farmers Market
2,East Toronto,1,Park,Pet Store,Italian Restaurant,Burrito Place,Burger Joint,Fish & Chips Shop,Sandwich Place,Liquor Store,Board Shop,Steakhouse
3,East Toronto,1,Diner,Bakery,Italian Restaurant,Café,Bar,Sushi Restaurant,American Restaurant,Coffee Shop,Boutique,Juice Bar
4,Central Toronto,1,Bus Line,Swim School,Women's Store,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,Central Toronto,1,Breakfast Spot,Park,Food & Drink Shop,Grocery Store,Gym,Hotel,Burger Joint,Clothing Store,Fish Market,Fish & Chips Shop
6,Central Toronto,1,Playground,Gym Pool,Park,Garden,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
7,Central Toronto,1,Coffee Shop,Pizza Place,Café,Italian Restaurant,Seafood Restaurant,Dessert Shop,Sandwich Place,Restaurant,Sushi Restaurant,Indian Restaurant
8,Central Toronto,1,Playground,Park,Gym,Tennis Court,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
9,Central Toronto,1,Coffee Shop,Light Rail Station,Convenience Store,Supermarket,Women's Store,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


### Cluster 3

In [178]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Downtown Toronto,2,Café,Coffee Shop,Restaurant,Bakery,Gym,Bookstore,Greek Restaurant,Japanese Restaurant,Thai Restaurant,Bar


### Cluster 4

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Plaza,Tea Room,Chinese Restaurant,Spa,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant
20,Downtown Toronto,3,Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Restaurant,Deli / Bodega,Bar,Asian Restaurant,Japanese Restaurant


### Cluster 5

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,4,Coffee Shop,Restaurant,Breakfast Spot,Electronics Store,Pub,Bakery,Event Space,Italian Restaurant,Food Truck,Spa
36,East Toronto,4,Coffee Shop,Bar,Café,Steakhouse,Restaurant,Japanese Restaurant,Hotel,American Restaurant,Thai Restaurant,Concert Hall
